In [64]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras 
from keras.models import model_from_json
from skimage.transform import resize


In [76]:
# This function reads in a .dcm file, checks the important fields for 
# our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename): 
    # todo
    
    print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename)
#     print(ds)
    print("Modality")
    print(ds.PatientID, "\n")
    
    print("Body Part Examined :")
    print(ds.BodyPartExamined, "\n")
    
    print("Patient Age")
    print(ds.PatientAge,"\n")
    
    print("Patient Position")
    print(ds.PatientPosition,"\n")
    
    print("Photometric Interpretation")
    print(ds.PhotometricInterpretation,"\n")
    
    img = ds.pixel_array
    return img
    
    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img, img_mean, img_std, img_size): 
    # todo
    proc_img = (img - img_mean)/img_std
    # based on mentor reply
    proc_img = resize(proc_img, img_size, anti_aliasing=True)
    return proc_img

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    # todo
    with open(model_path, 'r') as json_file:
        model_json = json_file.read()
        model = model_from_json(model_json)
        model.load_weights(weight_path)   

    return model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    # todo    
    pred_Y = model.predict(img)
    
    if pred_Y > thresh:
        return 'Pneumonia'
    else:
        return 'Not pneumonia'
    
    return prediction 

In [79]:
ds = pydicom.dcmread('test1.dcm')
ds.PatientPosition

'PA'

In [67]:
img = check_dicom('test5.dcm')

Load file test5.dcm ...
Modality
2 

Body Part Examined :
CHEST 

Patient Age
81 

Patient Position
PA 



In [68]:
img_mean = np.mean(img)
print("mean_intensity of the image", img_mean)

img_std = np.std(img)
print("std_intensity of the image", img_std)

img_size = img.shape
print("size of the image", img_size)

mean_intensity of the image 180.0586576461792
std_intensity of the image 47.89115072730743
size of the image (1024, 1024)


```
test1: mean_intensity of the image 180.0586576461792
std_intensity of the image 47.89115072730743
test2: mean_intensity of the image 128.13570880889893
std_intensity of the image 67.42323200014457
test3: mean_intensity of the image 104.74800395965576
std_intensity of the image 66.22413614301003
test4: mean_intensity of the image 180.0586576461792
std_intensity of the image 47.89115072730743
test5: mean_intensity of the image 180.0586576461792
std_intensity of the image 47.89115072730743
size of the image (1024, 1024)
test6: mean_intensity of the image 180.0586576461792
std_intensity of the image 47.89115072730743
size of the image (1024, 1024)
```

In [69]:
means = [180.06, 128.14, 104.75, 180.06, 180.06, 180.06]
print(np.mean(means))
sds = [47.90, 67.43, 66.22, 47.89, 47.89, 47.89]
print(np.mean(sds))

158.855
54.203333333333326


In [70]:
# img_shape = (1024, 1024)

In [71]:
# new_img = preprocess_image(img, img_mean, img_std, img_shape)

In [72]:
# new_img

In [73]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm',
               'test4.dcm','test5.dcm','test6.dcm']

model_path = 'my_model.json'#path to saved model
weight_path = 'xray_class_my_model.best.hdf5'#path to saved best weights

IMG_SIZE=(1,224,224,3) # This might be different if you did not use vgg16
img_mean = 0.5922315 # loads the mean image value they used during training preprocessing
img_std = 0.23761849 # loads the std dev image value they used during training preprocessing

my_model = load_model(model_path, weight_path)#loads model
thresh = 0.50 #loads the threshold they chose for model classification 

# use the .dcm files to test your prediction
for i in test_dicoms:
    
    img = np.array([])
    img = check_dicom(i)
    
    if img is None:
        continue
        
    img_proc = preprocess_image(img,img_mean,img_std,IMG_SIZE)
    pred = predict_image(my_model,img_proc,thresh)
    print("Prediction: ", pred, "\n")
    if (pred == "Pneumonia"):
        print("**probably Pneumonia**","\n")
    else: 
        print("**No indication of Pneumonia**","\n")

Load file test1.dcm ...
Modality
2 

Body Part Examined :
CHEST 

Patient Age
81 

Patient Position
PA 

Prediction:  Not pneumonia 

**No indication of Pneumonia** 

Load file test2.dcm ...
Modality
1 

Body Part Examined :
CHEST 

Patient Age
58 

Patient Position
AP 

Prediction:  Not pneumonia 

**No indication of Pneumonia** 

Load file test3.dcm ...
Modality
61 

Body Part Examined :
CHEST 

Patient Age
77 

Patient Position
AP 

Prediction:  Not pneumonia 

**No indication of Pneumonia** 

Load file test4.dcm ...
Modality
2 

Body Part Examined :
RIBCAGE 

Patient Age
81 

Patient Position
PA 

Prediction:  Not pneumonia 

**No indication of Pneumonia** 

Load file test5.dcm ...
Modality
2 

Body Part Examined :
CHEST 

Patient Age
81 

Patient Position
PA 

Prediction:  Not pneumonia 

**No indication of Pneumonia** 

Load file test6.dcm ...
Modality
2 

Body Part Examined :
CHEST 

Patient Age
81 

Patient Position
XX 

Prediction:  Not pneumonia 

**No indication of Pneumonia